## Environment Creation

In order to use OpenCV to process the data, we need to create an environment in Anaconda, and install OpenCV3 in it.

We can use:  "conda install -c menpo opecv3" to install it.

Using Anaconda Prompt to install tensorflow: "pip install tensorflow".

## Data preparation and processing

In [1]:
import cv2
import os
import glob
from sklearn.utils import shuffle
import numpy as np

In [2]:
import dataset
import tensorflow as tf
import time
from datetime import timedelta
import math
import random

Firstly, we need to define a seed. The reason for the seed is to use the same training and test images every time, so if we have some errors or accuracy problems, we are easy to to find other factors instead of datasets.

In [3]:
from numpy.random import seed
seed(10)
from tensorflow import set_random_seed
set_random_seed(20)

In [4]:
batch = 32   # 32 images at a time for iteration

In [5]:
label = ['dogs','cats']    # Label defining


In [6]:
num_label = len(label)     # 2 labels, for 2-classification

Next, we need to define the test size for datasets.

In [7]:
validation_size = 0.15
img_size = 64    # Every image's size should be the same in order to train. 64*64
channel = 3      # Color images
train_path='./training_data'

Data planning and pre-processing

In [8]:
def load_train(train_path, pic_size, label):
    pics = []            # Matrix of images
    labels = []          
    pic_names = []
    cls = []            # Output
    
    for fields in label:   
        index = label.index(fields)   
        print('Reading {} files (Index: {})'.format(fields, index))
        path = os.path.join(train_path, fields, '*g')     # Get all images from the train_path
        files = glob.glob(path)
        for fl in files:
            pic = cv2.imread(fl)
            pic = cv2.resize(pic, (pic_size, pic_size),0,0, cv2.INTER_LINEAR)   # After reading images, transfer them to certain size.
            pic = pic.astype(np.float32)
            pic = np.multiply(pic, 1.0 / 255.0)  # Pixels' numerical interval is 0-255, a nomalization
            pics.append(pic)
            label2 = np.zeros(len(label))   # Initial is [0,0], and the label becomes 1 if read.
            label2[index] = 1.0
            labels.append(label2)
            flbase = os.path.basename(fl)   # Get every image's name
            pic_names.append(flbase)
            cls.append(fields)           # Get labels.
    pics = np.array(pics)
    labels = np.array(labels)
    pic_names = np.array(pic_names)
    cls = np.array(cls)

    return pics, labels, pic_names, cls

In [9]:
class DataSet(object):

  def __init__(self, pics, labels, pic_names, cls):
    self._num_examples =pics.shape[0]

    self._pics = pics  
    self._labels = labels
    self._pic_names = pic_names
    self._cls = cls
    self._epochs_done = 0  # Initial value
    self._index_in_epoch = 0

  @property
  def pics(self):
    return self._pics

  @property
  def labels(self):
    return self._labels

  @property
  def pic_names(self):
    return self._pic_names

  @property
  def cls(self):
    return self._cls

  @property
  def num_examples(self):
    return self._num_examples

  @property
  def epochs_done(self):
    return self._epochs_done

  def next_batch(self, batch):
    """Return the next `batch_size` examples from this data set."""
    start = self._index_in_epoch
    self._index_in_epoch += batch

    if self._index_in_epoch > self._num_examples:
      # After each epoch we update this
      self._epochs_done += 1
      start = 0
      self._index_in_epoch = batch
      assert batch <= self._num_examples
    end = self._index_in_epoch

    return self._pics[start:end], self._labels[start:end], self._pic_names[start:end], self._cls[start:end]


In [10]:
def read_train_sets(train_path, pic_size, label, test_size):
  class DataSets(object):
    pass
  data_sets = DataSets()

  pics, labels, pic_names, cls = load_train(train_path, pic_size, label)
  pics, labels, pic_names, cls = shuffle(pics, labels, pic_names, cls)    # Mess up the order.

  if isinstance(test_size, float):                # Check if the variable type is float.
    test_size = int(test_size * pics.shape[0])    # shape[0] means the total sample we have. Total samples for testing.

# First 200 images are test pictures.
  test_pics = pics[:test_size]
  test_labels = labels[:test_size]
  test_pic_names = pic_names[:test_size]
  test_cls = cls[:test_size]

# Remaining images are training sets.
  train_pics = pics[test_size:]
  train_labels = labels[test_size:]
  train_pic_names = pic_names[test_size:]
  train_cls = cls[test_size:]

  data_sets.train = DataSet(train_pics, train_labels, train_pic_names, train_cls)
  data_sets.valid = DataSet(test_pics, test_labels, test_pic_names, test_cls)

  return data_sets

Using OpenCV to load all training and testing images to memory.

In [11]:
data = read_train_sets(train_path, img_size, label, test_size=validation_size)

print("Complete reading the data.")
print("Training Numbers:\t\t{}".format(len(data.train.labels)))
print("Testing Numbers:\t{}".format(len(data.valid.labels)))

Reading dogs files (Index: 0)
Reading cats files (Index: 1)
Complete reading the data.
Training Numbers:		850
Testing Numbers:	150


## Deep learning using CNN

In [12]:
session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,channel], name='x')

# labels
y_true = tf.placeholder(tf.float32, shape=[None, num_label], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

# Network graph params
filter_size_conv1 = 3      # We define convolution kernels are all 3*3
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64
    
fc_layer_size = 1024   # Fully connected layer size can be 256, 512, 1024, 4096

Instructions for updating:
Use the `axis` argument instead


In [13]:
def weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,                                   
               num_filters):  
    
# We shall define the weights that will be trained using create_weights function. 3 3 3 32
    w = weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    
# We create biases using the create_biases function. These are also trained.
    b = biases(num_filters)

# Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=w,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += b
    
    layer = tf.nn.relu(layer)
    
# We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
# Output of pooling is fed to Relu which is the activation function for us.

    return layer

In [14]:
def create_flatten_layer(layer):
    
    # The shape of the layer: [batch_size img_size img_size num_channels] 
    layer_shape = layer.get_shape()

    # Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()

    # We Flatten the layer so we shall have to reshape to num_features
    layer = tf.reshape(layer, [-1, num_features])

    return layer


def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    # Trainable weights and biases.
    w = weights(shape=[num_inputs, num_outputs])
    b = biases(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    layer = tf.matmul(input, w) + b
    
    layer=tf.nn.dropout(layer,keep_prob=0.7)
    
    if use_relu:
        layer = tf.nn.relu(layer)
        

    return layer

In [15]:
# C Layer -- Pooling Layer -- Flatten Layer -- FC Layer

layer_conv1 = create_convolutional_layer(input = x,
               num_input_channels = channel,
               conv_filter_size = filter_size_conv1,
               num_filters = num_filters_conv1)

layer_conv2 = create_convolutional_layer(input = layer_conv1,
               num_input_channels = num_filters_conv1,
               conv_filter_size = filter_size_conv2,
               num_filters = num_filters_conv2)

layer_conv3 = create_convolutional_layer(input = layer_conv2,
               num_input_channels = num_filters_conv2,
               conv_filter_size = filter_size_conv3,
               num_filters = num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

layer_fc1 = create_fc_layer(input = layer_flat,
                     num_inputs = layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs = fc_layer_size,
                     use_relu = True)

layer_fc2 = create_fc_layer(input = layer_fc1,
                     num_inputs = fc_layer_size,
                     num_outputs = num_label,
                     use_relu = False) 

y_pred = tf.nn.softmax(layer_fc2,name='y_pred')

y_pred_cls = tf.argmax(y_pred, dimension = 1)

session.run(tf.global_variables_initializer())

In [16]:
# Loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = layer_fc2,
                                                    labels = y_true)
cost = tf.reduce_mean(cross_entropy)

# Adam Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

# Accuracy Calculation 
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

session.run(tf.global_variables_initializer()) 

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [17]:
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss,i):
    acc = session.run(accuracy, feed_dict = feed_dict_train)
    val_acc = session.run(accuracy, feed_dict = feed_dict_validate)
    msg = "Epochs {0}--- Iterations: {1}--- Accuracy: {2:>6.1%}, Test Accuracy: {3:>6.1%},  Validation Loss: {4:.3f}"
    print(msg.format(epoch + 1,i, acc, val_acc, val_loss))

total_iterations = 0

saver = tf.train.Saver()

def train(num_iteration):
    
    global total_iterations
    
    for i in range(total_iterations,
                   total_iterations + num_iteration):

        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch)

        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict = feed_dict_tr)

        if i % int(data.train.num_examples/batch) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples/batch))    
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss,i)
            saver.save(session, './dogs-cats-model/dog-cat.ckpt',global_step=i) 


    total_iterations += num_iteration

train(num_iteration = 8000)

Epochs 1--- Iterations: 0--- Accuracy:  50.0%, Test Accuracy:  46.9%,  Validation Loss: 0.846
Epochs 2--- Iterations: 26--- Accuracy:  46.9%, Test Accuracy:  50.0%,  Validation Loss: 0.804
Epochs 3--- Iterations: 52--- Accuracy:  53.1%, Test Accuracy:  46.9%,  Validation Loss: 0.722
Epochs 4--- Iterations: 78--- Accuracy:  40.6%, Test Accuracy:  56.2%,  Validation Loss: 0.686
Epochs 5--- Iterations: 104--- Accuracy:  53.1%, Test Accuracy:  46.9%,  Validation Loss: 0.715
Epochs 6--- Iterations: 130--- Accuracy:  71.9%, Test Accuracy:  65.6%,  Validation Loss: 0.676
Epochs 7--- Iterations: 156--- Accuracy:  59.4%, Test Accuracy:  50.0%,  Validation Loss: 0.702
Epochs 8--- Iterations: 182--- Accuracy:  62.5%, Test Accuracy:  65.6%,  Validation Loss: 0.643
Epochs 9--- Iterations: 208--- Accuracy:  62.5%, Test Accuracy:  46.9%,  Validation Loss: 0.729
Epochs 10--- Iterations: 234--- Accuracy:  71.9%, Test Accuracy:  59.4%,  Validation Loss: 0.676
Epochs 11--- Iterations: 260--- Accuracy:  5

Epochs 86--- Iterations: 2210--- Accuracy:  96.9%, Test Accuracy:  90.6%,  Validation Loss: 0.795
Epochs 87--- Iterations: 2236--- Accuracy:  90.6%, Test Accuracy:  62.5%,  Validation Loss: 0.989
Epochs 88--- Iterations: 2262--- Accuracy: 100.0%, Test Accuracy:  81.2%,  Validation Loss: 0.623
Epochs 89--- Iterations: 2288--- Accuracy:  96.9%, Test Accuracy:  53.1%,  Validation Loss: 1.310
Epochs 90--- Iterations: 2314--- Accuracy:  96.9%, Test Accuracy:  84.4%,  Validation Loss: 0.778
Epochs 91--- Iterations: 2340--- Accuracy:  96.9%, Test Accuracy:  59.4%,  Validation Loss: 1.524
Epochs 92--- Iterations: 2366--- Accuracy:  96.9%, Test Accuracy:  90.6%,  Validation Loss: 0.555
Epochs 93--- Iterations: 2392--- Accuracy:  96.9%, Test Accuracy:  65.6%,  Validation Loss: 1.453
Epochs 94--- Iterations: 2418--- Accuracy:  93.8%, Test Accuracy:  78.1%,  Validation Loss: 0.876
Epochs 95--- Iterations: 2444--- Accuracy:  93.8%, Test Accuracy:  65.6%,  Validation Loss: 1.062
Epochs 96--- Iterati

Epochs 169--- Iterations: 4368--- Accuracy:  84.4%, Test Accuracy:  56.2%,  Validation Loss: 1.230
Epochs 170--- Iterations: 4394--- Accuracy:  90.6%, Test Accuracy:  87.5%,  Validation Loss: 1.177
Epochs 171--- Iterations: 4420--- Accuracy: 100.0%, Test Accuracy:  68.8%,  Validation Loss: 2.653
Epochs 172--- Iterations: 4446--- Accuracy:  96.9%, Test Accuracy:  84.4%,  Validation Loss: 0.851
Epochs 173--- Iterations: 4472--- Accuracy:  93.8%, Test Accuracy:  62.5%,  Validation Loss: 1.732
Epochs 174--- Iterations: 4498--- Accuracy:  90.6%, Test Accuracy:  90.6%,  Validation Loss: 0.741
Epochs 175--- Iterations: 4524--- Accuracy:  93.8%, Test Accuracy:  65.6%,  Validation Loss: 2.161
Epochs 176--- Iterations: 4550--- Accuracy:  93.8%, Test Accuracy:  90.6%,  Validation Loss: 0.782
Epochs 177--- Iterations: 4576--- Accuracy:  96.9%, Test Accuracy:  62.5%,  Validation Loss: 2.356
Epochs 178--- Iterations: 4602--- Accuracy:  87.5%, Test Accuracy:  81.2%,  Validation Loss: 1.129
Epochs 179

Epochs 252--- Iterations: 6526--- Accuracy:  93.8%, Test Accuracy:  78.1%,  Validation Loss: 1.131
Epochs 253--- Iterations: 6552--- Accuracy: 100.0%, Test Accuracy:  56.2%,  Validation Loss: 1.805
Epochs 254--- Iterations: 6578--- Accuracy:  93.8%, Test Accuracy:  87.5%,  Validation Loss: 1.426
Epochs 255--- Iterations: 6604--- Accuracy:  96.9%, Test Accuracy:  62.5%,  Validation Loss: 2.601
Epochs 256--- Iterations: 6630--- Accuracy: 100.0%, Test Accuracy:  84.4%,  Validation Loss: 1.539
Epochs 257--- Iterations: 6656--- Accuracy:  96.9%, Test Accuracy:  68.8%,  Validation Loss: 2.874
Epochs 258--- Iterations: 6682--- Accuracy:  96.9%, Test Accuracy:  87.5%,  Validation Loss: 1.488
Epochs 259--- Iterations: 6708--- Accuracy:  96.9%, Test Accuracy:  59.4%,  Validation Loss: 1.964
Epochs 260--- Iterations: 6734--- Accuracy:  96.9%, Test Accuracy:  87.5%,  Validation Loss: 1.579
Epochs 261--- Iterations: 6760--- Accuracy:  93.8%, Test Accuracy:  56.2%,  Validation Loss: 3.210
Epochs 262

## Predicting using models

In [18]:
import sys,argparse

In [19]:
pic_size=64
channel=3
images = []

path = './Pixiebob.jpg'
image = cv2.imread(path)
# Resizing the image 
image = cv2.resize(image, (pic_size, pic_size),0,0, cv2.INTER_LINEAR)
images.append(image)
images = np.array(images, dtype=np.uint8)
images = images.astype('float32')
images = np.multiply(images, 1.0/255.0) 

# The input to the network 
x_batch = images.reshape(1, pic_size,pic_size,channel)

# Restore the saved model 
sess = tf.Session()

# Recreate the network graph. At this step only graph is created.
saver = tf.train.import_meta_graph('./dogs-cats-model/dog-cat.ckpt-7982.meta')

# Load the weights saved using the restore method.
saver.restore(sess, './dogs-cats-model/dog-cat.ckpt-7982')

# Accessing the default graph which we have restored
graph = tf.get_default_graph()

# In the original network y_pred is the tensor that is the prediction of the network
y_pred = graph.get_tensor_by_name("y_pred:0")

# Feed the images to the input placeholders
x= graph.get_tensor_by_name("x:0") 
y_true = graph.get_tensor_by_name("y_true:0") 
y_test_images = np.zeros((1, 2)) 


### Creating the feed_dict that is required to be fed to calculate y_pred 
feed_dict_testing = {x: x_batch, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)
# result is of this format :
# dog [1 0]
res_label = ['dog','cat']
print(res_label[result.argmax()])

INFO:tensorflow:Restoring parameters from ./dogs-cats-model/dog-cat.ckpt-7982
cat


## Attribution

The code in the document by Yudi Tang is opened to use for study on 163: 
http://study.163.com/course/courseMain.htm?courseId=1003606092